# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 7 2022 10:55AM,250162,10,MSP215517,"Methapharm, Inc.",Released
1,Nov 7 2022 10:55AM,250162,10,MSP215536,"Methapharm, Inc.",Released
2,Nov 7 2022 10:55AM,250162,10,MSP215537,"Methapharm, Inc.",Released
3,Nov 7 2022 10:55AM,250162,10,MSP215558,"Methapharm, Inc.",Released
4,Nov 7 2022 10:55AM,250162,10,MSP215572,"Methapharm, Inc.",Released
5,Nov 7 2022 10:51AM,250157,19,8745074,"Graystone, LLC",Released
6,Nov 7 2022 10:51AM,250157,19,8745075,"Graystone, LLC",Released
7,Nov 7 2022 10:51AM,250157,19,8745082,"Graystone, LLC",Released
8,Nov 7 2022 10:51AM,250161,10,MSP215510,"Methapharm, Inc.",Released
9,Nov 7 2022 10:51AM,250161,10,MSP215516,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,250156,Released,4
19,250157,Released,3
20,250160,Released,1
21,250161,Released,7
22,250162,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250156,NaN,4.0
250157,NaN,3.0
250160,NaN,1.0
250161,NaN,7.0
250162,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250118,3.0,3.0
250121,1.0,0.0
250130,0.0,1.0
250131,0.0,1.0
250141,2.0,28.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250118,3,3
250121,1,0
250130,0,1
250131,0,1
250141,2,28


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250118,3,3
1,250121,1,0
2,250130,0,1
3,250131,0,1
4,250141,2,28


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250118,3,3
1,250121,1,
2,250130,,1
3,250131,,1
4,250141,2,28


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 7 2022 10:55AM,250162,10,"Methapharm, Inc."
5,Nov 7 2022 10:51AM,250157,19,"Graystone, LLC"
8,Nov 7 2022 10:51AM,250161,10,"Methapharm, Inc."
15,Nov 7 2022 10:50AM,250160,19,"Avet Pharmaceuticals Labs, Inc"
16,Nov 7 2022 10:50AM,250149,10,Eywa Pharma Inc.
21,Nov 7 2022 10:47AM,250156,16,Sartorius Bioprocess Solutions
25,Nov 7 2022 10:47AM,250155,15,"Person & Covey, Inc."
92,Nov 7 2022 10:41AM,250154,10,"Virtue Rx, LLC"
93,Nov 7 2022 10:32AM,250151,10,ISDIN Corporation
94,Nov 7 2022 10:31AM,250150,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 7 2022 10:55AM,250162,10,"Methapharm, Inc.",,5
1,Nov 7 2022 10:51AM,250157,19,"Graystone, LLC",,3
2,Nov 7 2022 10:51AM,250161,10,"Methapharm, Inc.",,7
3,Nov 7 2022 10:50AM,250160,19,"Avet Pharmaceuticals Labs, Inc",,1
4,Nov 7 2022 10:50AM,250149,10,Eywa Pharma Inc.,,5
5,Nov 7 2022 10:47AM,250156,16,Sartorius Bioprocess Solutions,,4
6,Nov 7 2022 10:47AM,250155,15,"Person & Covey, Inc.",,67
7,Nov 7 2022 10:41AM,250154,10,"Virtue Rx, LLC",,1
8,Nov 7 2022 10:32AM,250151,10,ISDIN Corporation,,1
9,Nov 7 2022 10:31AM,250150,21,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 10:55AM,250162,10,"Methapharm, Inc.",5,
1,Nov 7 2022 10:51AM,250157,19,"Graystone, LLC",3,
2,Nov 7 2022 10:51AM,250161,10,"Methapharm, Inc.",7,
3,Nov 7 2022 10:50AM,250160,19,"Avet Pharmaceuticals Labs, Inc",1,
4,Nov 7 2022 10:50AM,250149,10,Eywa Pharma Inc.,5,
5,Nov 7 2022 10:47AM,250156,16,Sartorius Bioprocess Solutions,4,
6,Nov 7 2022 10:47AM,250155,15,"Person & Covey, Inc.",67,
7,Nov 7 2022 10:41AM,250154,10,"Virtue Rx, LLC",1,
8,Nov 7 2022 10:32AM,250151,10,ISDIN Corporation,1,
9,Nov 7 2022 10:31AM,250150,21,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 10:55AM,250162,10,"Methapharm, Inc.",5,
1,Nov 7 2022 10:51AM,250157,19,"Graystone, LLC",3,
2,Nov 7 2022 10:51AM,250161,10,"Methapharm, Inc.",7,
3,Nov 7 2022 10:50AM,250160,19,"Avet Pharmaceuticals Labs, Inc",1,
4,Nov 7 2022 10:50AM,250149,10,Eywa Pharma Inc.,5,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 10:55AM,250162,10,"Methapharm, Inc.",5.0,NaN
1,Nov 7 2022 10:51AM,250157,19,"Graystone, LLC",3.0,NaN
2,Nov 7 2022 10:51AM,250161,10,"Methapharm, Inc.",7.0,NaN
3,Nov 7 2022 10:50AM,250160,19,"Avet Pharmaceuticals Labs, Inc",1.0,NaN
4,Nov 7 2022 10:50AM,250149,10,Eywa Pharma Inc.,5.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 10:55AM,250162,10,"Methapharm, Inc.",5.0,0.0
1,Nov 7 2022 10:51AM,250157,19,"Graystone, LLC",3.0,0.0
2,Nov 7 2022 10:51AM,250161,10,"Methapharm, Inc.",7.0,0.0
3,Nov 7 2022 10:50AM,250160,19,"Avet Pharmaceuticals Labs, Inc",1.0,0.0
4,Nov 7 2022 10:50AM,250149,10,Eywa Pharma Inc.,5.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2001203,51.0,2.0
15,250155,67.0,0.0
16,750407,5.0,1.0
18,250131,1.0,0.0
19,1250725,17.0,3.0
21,1000591,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2001203,51.0,2.0
1,15,250155,67.0,0.0
2,16,750407,5.0,1.0
3,18,250131,1.0,0.0
4,19,1250725,17.0,3.0
5,21,1000591,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,51.0,2.0
1,15,67.0,0.0
2,16,5.0,1.0
3,18,1.0,0.0
4,19,17.0,3.0
5,21,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,51.0
1,15,Released,67.0
2,16,Released,5.0
3,18,Released,1.0
4,19,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,21
Status,,,,,,
Executing,2.0,0.0,1.0,0.0,3.0,0.0
Released,51.0,67.0,5.0,1.0,17.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,21
0,Executing,2.0,0.0,1.0,0.0,3.0,0.0
1,Released,51.0,67.0,5.0,1.0,17.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,21
0,Executing,2.0,0.0,1.0,0.0,3.0,0.0
1,Released,51.0,67.0,5.0,1.0,17.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()